In [1]:
#Import library pandas dan membaca data
import pandas as pd
data = pd.read_csv('/content/sample_data/Tweets.csv', sep=',')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [2]:
data = data[['text','airline_sentiment']]
data = data[data['airline_sentiment'] != 'neutral']
data.head()

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [3]:
#Ubah sentiment menjadi numeric agar bisa dibaca oleh model machine learning nantinya
label = data.airline_sentiment.factorize()
label

(array([0, 1, 1, ..., 0, 1, 1]),
 Index(['positive', 'negative'], dtype='object'))

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tweet = data.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [26]:
print(tweet[0],'\n')
print(encoded_docs[0])

@VirginAmerica plus you've added commercials to the experience... tacky. 

[103, 575, 530, 1287, 2416, 1, 2, 177]


In [6]:
# Build the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 64
model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_vector_length, input_length=200),
        tf.keras.layers.SpatialDropout1D(0.25),
        tf.keras.layers.LSTM(50, dropout=0.5, recurrent_dropout=0.5),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 64)           846976    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                23000     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 870,027
Trainable params: 870,027
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
history = model.fit(padded_sequence,label[0],validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
289/289 [==============================] - 229s 778ms/step - loss: 0.4818 - accuracy: 0.7945 - val_loss: 0.1992 - val_accuracy: 0.9242
Epoch 2/5
289/289 [==============================] - 221s 766ms/step - loss: 0.2080 - accuracy: 0.9191 - val_loss: 0.1707 - val_accuracy: 0.9398
Epoch 3/5
289/289 [==============================] - 217s 751ms/step - loss: 0.1337 - accuracy: 0.9507 - val_loss: 0.1745 - val_accuracy: 0.9381
Epoch 4/5
289/289 [==============================] - 215s 744ms/step - loss: 0.1111 - accuracy: 0.9577 - val_loss: 0.1741 - val_accuracy: 0.9459
Epoch 5/5
289/289 [==============================] - 220s 760ms/step - loss: 0.0831 - accuracy: 0.9698 - val_loss: 0.1846 - val_accuracy: 0.9385


In [36]:
test1 ='Anything in here is not positive.'
test2 ='Im so happy right now.'
tw = tokenizer.texts_to_sequences([test1])
tw

[[292, 15, 148, 14, 23, 1749]]

In [37]:
tw = pad_sequences(tw, maxlen=200)
tw

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [38]:
#TEST 1
prediction = int(model.predict(tw).round().item())
label[1][prediction]

'negative'

In [35]:
#TEST 2
prediction = int(model.predict(tw).round().item())
label[1][prediction]

'positive'